In [48]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
sns.set()
pd.options.display.max_rows = 100

df = pd.read_csv('CR_FC_PREP_0522.csv')

df.head()
df.info()
print(f'Dataset size: {df.shape}')

,POD,RATE
0,2005-07-19,2200.0
1,2005-07-20,2250.0
2,2005-07-21,2300.0
3,2005-07-22,2280.0
4,2005-07-23,2260.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6375 entries, 0 to 6374
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   POD     6375 non-null   object 
 1   RATE    6375 non-null   float64
dtypes: float64(1), object(1)
memory usage: 99.7+ KB
Dataset size: (6375, 2)


In [49]:
df['POD'] = pd.to_datetime(df['POD'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6375 entries, 0 to 6374
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   POD     6375 non-null   datetime64[ns]
 1   RATE    6375 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 99.7 KB


In [50]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Y is the target
Y = df.pop('RATE') 
X = df

# Normalise and split the data into training and testing sets
sc = StandardScaler()
X = sc.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [51]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

def optimize_decision_tree(X_train, X_test, y_train, y_test):
    # Define the hyperparameter grid to search over
    param_grid = {'max_depth': [3, 5, 7, 9, 11],
                  'min_samples_split': [2, 5, 10, 20],
                  'min_samples_leaf': [1, 2, 4, 8]}
    
    
    # Create a decision tree regressor object
    dt = DecisionTreeRegressor()
    
    # Perform grid search to find the best hyperparameters
    grid_search = GridSearchCV(dt, param_grid, cv=5)
    grid_search.fit(X_train, y_train)
    
    # Print the best hyperparameters and their corresponding R^2 score
    print("Best hyperparameters:", grid_search.best_params_)
    y_pred = grid_search.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    print("R^2 score on test data:", r2)
    
    # Return the optimized decision tree model
    return grid_search.best_estimator_


In [52]:
optimize_decision_tree(X_train, X_test, y_train, y_test)

Best hyperparameters: {'max_depth': 11, 'min_samples_leaf': 1, 'min_samples_split': 2}
R^2 score on test data: 0.9944387220292452


DecisionTreeRegressor(max_depth=11)

In [53]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score

# Train a decision tree model with the best hyperparameters found
model = DecisionTreeRegressor(max_depth=11, min_samples_leaf=1, min_samples_split=2)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate root mean square error, R-squared value, and accuracy
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
mean = y_test.mean()
acc = (1 - rmse/mean) * 100
    
# Print the results
print("Root mean squared error: {:.3f}".format(rmse))
print("R-squared value: {:.3f}".format(r2))
print("Accuracy: {:.2f}%".format(acc))

DecisionTreeRegressor(max_depth=11)

Root mean squared error: 148.341
R-squared value: 0.994
Accuracy: 90.74%
